In [48]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
import math
import torch
import time
import re
import cv2
import shutil
from pathlib import Path
from tqdm import tqdm
from matplotlib import pyplot as plt
from torchvision import transforms
from icecream import ic
import tensorflow as tf 
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)
np.set_printoptions(precision=2,suppress=True)

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(device_type='GPU', devices=gpus[0])

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [30]:
maindf = pd.read_csv("./Sources/TSLA.csv")
maindf.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-02-17,205.699997,205.699997,201.500000,204.350006,204.350006,3979600
1,2015-02-18,204.169998,206.169998,202.600006,204.460007,204.460007,2713600
2,2015-02-19,205.000000,212.440002,203.750000,211.710007,211.710007,5154100
3,2015-02-20,210.779999,217.600006,209.809998,217.110001,217.110001,5982100
4,2015-02-23,215.660004,218.199997,206.330002,207.339996,207.339996,8499800


In [31]:
maindf = maindf.rename(
    columns={
        'Date':'date',
        'Open':'open',
        'High':'high',
        'Low':'low',
        'Close':'close',
        'Adj Close':'adj_close',
        'Volume':'volume'
    }
)
maindf.head()

,date,open,high,low,close,adj_close,volume
0,2015-02-17,205.699997,205.699997,201.500000,204.350006,204.350006,3979600
1,2015-02-18,204.169998,206.169998,202.600006,204.460007,204.460007,2713600
2,2015-02-19,205.000000,212.440002,203.750000,211.710007,211.710007,5154100
3,2015-02-20,210.779999,217.600006,209.809998,217.110001,217.110001,5982100
4,2015-02-23,215.660004,218.199997,206.330002,207.339996,207.339996,8499800


In [32]:
maindf.isnull().values.sum()
maindf.dropna()

,date,open,high,low,close,adj_close,volume
0,2015-02-17,205.699997,205.699997,201.500000,204.350006,204.350006,3979600
1,2015-02-18,204.169998,206.169998,202.600006,204.460007,204.460007,2713600
2,2015-02-19,205.000000,212.440002,203.750000,211.710007,211.710007,5154100
3,2015-02-20,210.779999,217.600006,209.809998,217.110001,217.110001,5982100
4,2015-02-23,215.660004,218.199997,206.330002,207.339996,207.339996,8499800
...,...,...,...,...,...,...,...
1254,2020-02-10,800.000000,819.989990,752.400024,771.280029,771.280029,24689200
1255,2020-02-11,768.789978,783.510010,758.000000,774.380005,774.380005,11697500
1256,2020-02-12,777.869995,789.750000,763.369995,767.289978,767.289978,12022500
1257,2020-02-13,741.840027,818.000000,735.000000,804.000000,804.000000,26289300


In [33]:
maindf['date'] = pd.to_datetime(maindf.date)
maindf.head()

,date,open,high,low,close,adj_close,volume
0,2015-02-17,205.699997,205.699997,201.500000,204.350006,204.350006,3979600
1,2015-02-18,204.169998,206.169998,202.600006,204.460007,204.460007,2713600
2,2015-02-19,205.000000,212.440002,203.750000,211.710007,211.710007,5154100
3,2015-02-20,210.779999,217.600006,209.809998,217.110001,217.110001,5982100
4,2015-02-23,215.660004,218.199997,206.330002,207.339996,207.339996,8499800


In [34]:
# 使用股票调整价计算滑动平均价格(Adjusted Closing Price)
maindf['moving_avg']=maindf['adj_close'].rolling(window=20, min_periods=0).mean()
maindf.head()

,date,open,high,low,close,adj_close,volume,moving_avg
0,2015-02-17,205.699997,205.699997,201.500000,204.350006,204.350006,3979600,204.350006
1,2015-02-18,204.169998,206.169998,202.600006,204.460007,204.460007,2713600,204.405007
2,2015-02-19,205.000000,212.440002,203.750000,211.710007,211.710007,5154100,206.840007
3,2015-02-20,210.779999,217.600006,209.809998,217.110001,217.110001,5982100,209.407505
4,2015-02-23,215.660004,218.199997,206.330002,207.339996,207.339996,8499800,208.994003


In [35]:
maindf.tail()

,date,open,high,low,close,adj_close,volume,moving_avg
1254,2020-02-10,800.000000,819.989990,752.400024,771.280029,771.280029,24689200,626.320505
1255,2020-02-11,768.789978,783.510010,758.000000,774.380005,774.380005,11697500,638.796506
1256,2020-02-12,777.869995,789.750000,763.369995,767.289978,767.289978,12022500,650.265005
1257,2020-02-13,741.840027,818.000000,735.000000,804.000000,804.000000,26289300,664.540005
1258,2020-02-14,787.219971,812.969971,785.500000,800.030029,800.030029,15656900,678.867007


In [36]:
len(maindf)

1259

In [37]:
# 计算波动率指数(Volatility indexes, VIX)
# 通常波动率指数 moving_avg的上下3%
i = 1
upper_volatility = [maindf.iloc[0]['moving_avg']]
lower_volatility = [maindf.iloc[0]['moving_avg']]
while i < len(maindf):
    upper_volatility.append(
        maindf.iloc[i-1]['moving_avg']+3/100*maindf.iloc[i-1]['moving_avg']
    )
    lower_volatility.append(
        maindf.iloc[i-1]['moving_avg']-3/100*maindf.iloc[i-1]['moving_avg']
    )
    i+=1
len(upper_volatility), len(lower_volatility)

(1259, 1259)

In [38]:
maindf['upper_volatility'] = upper_volatility
maindf['lower_volatility'] = lower_volatility

In [39]:
maindf

,date,open,high,low,close,adj_close,volume,moving_avg,upper_volatility,lower_volatility
0,2015-02-17,205.699997,205.699997,201.500000,204.350006,204.350006,3979600,204.350006,204.350006,204.350006
1,2015-02-18,204.169998,206.169998,202.600006,204.460007,204.460007,2713600,204.405007,210.480506,198.219506
2,2015-02-19,205.000000,212.440002,203.750000,211.710007,211.710007,5154100,206.840007,210.537157,198.272856
3,2015-02-20,210.779999,217.600006,209.809998,217.110001,217.110001,5982100,209.407505,213.045207,200.634806
4,2015-02-23,215.660004,218.199997,206.330002,207.339996,207.339996,8499800,208.994003,215.689730,203.125280
...,...,...,...,...,...,...,...,...,...,...
1254,2020-02-10,800.000000,819.989990,752.400024,771.280029,771.280029,24689200,626.320505,630.013923,593.314083
1255,2020-02-11,768.789978,783.510010,758.000000,774.380005,774.380005,11697500,638.796506,645.110120,607.530890
1256,2020-02-12,777.869995,789.750000,763.369995,767.289978,767.289978,12022500,650.265005,657.960401,619.632611
1257,2020-02-13,741.840027,818.000000,735.000000,804.000000,804.000000,26289300,664.540005,669.772956,630.757055


In [40]:
# 短期压力价格
maindf['short_resistance'] = maindf['high'].rolling(window=10, min_periods=0).max()

In [41]:
# 短期支撑价格
maindf['short_support'] = maindf['low'].rolling(window=10, min_periods=0).min()

In [42]:
# 长期压力价格
maindf['long_resistance'] = maindf['high'].rolling(window=50, min_periods=0).max()

In [43]:
# 长期支撑价格
maindf['long_support'] = maindf['low'].rolling(window=50, min_periods=0).min()

In [44]:
maindf.head()

,date,open,high,low,close,adj_close,volume,moving_avg,upper_volatility,lower_volatility,short_resistance,short_support,long_resistance,long_support
0,2015-02-17,205.699997,205.699997,201.500000,204.350006,204.350006,3979600,204.350006,204.350006,204.350006,205.699997,201.5,205.699997,201.5
1,2015-02-18,204.169998,206.169998,202.600006,204.460007,204.460007,2713600,204.405007,210.480506,198.219506,206.169998,201.5,206.169998,201.5
2,2015-02-19,205.000000,212.440002,203.750000,211.710007,211.710007,5154100,206.840007,210.537157,198.272856,212.440002,201.5,212.440002,201.5
3,2015-02-20,210.779999,217.600006,209.809998,217.110001,217.110001,5982100,209.407505,213.045207,200.634806,217.600006,201.5,217.600006,201.5
4,2015-02-23,215.660004,218.199997,206.330002,207.339996,207.339996,8499800,208.994003,215.689730,203.125280,218.199997,201.5,218.199997,201.5


In [28]:
from itertools import cycle
import plotly.express as px
names = cycle(["stock adjust price", "stock upper volatility", "stock lower volatility", "stock short support price", "stock short resistance"])


In [49]:
fig = px.line(
    maindf, 
    x=maindf.date,
    y=[
        maindf['adj_close'],
        maindf['upper_volatility'],
        maindf['lower_volatility'],
        maindf['short_support'],
        maindf['short_resistance'],
    ],
    labels={'date':'date','value':"stock price"},
)
fig.update_layout(
    title_text='stock sequence analysis',
    font_size=15,
    font_color='black'
)
fig.for_each_trace(lambda t:t.update(name=next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()